In [154]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime
from concurrent import futures
from pandas import DataFrame,read_csv,concat
import scrap_config 

result_list=[]
def exclude_checked_keywords(new_keywords_set):
    print("Excluding already checked keywords")
    print("New Word List Length-->",len(new_keywords_set))
    done_keywords_df=read_csv(scrap_config.done_keywords_csv_path)
    done_keywords_df.columns=map(str.lower,done_keywords_df.columns)
    checked_keyword_set=set(done_keywords_df.keyword)
    remaining_keywords_list=list(new_keywords_set-checked_keyword_set)
    print("Number of checked keywords->",len(checked_keyword_set))
    print("Number of unchecked keywords->",len(remaining_keywords_list))
    return remaining_keywords_list
    # return checked_keyword_set,checked_keyword_set

def update_done_keyword_csv(df:DataFrame):
    df=df[["keyword","checking_date"]]
    base_df=read_csv(scrap_config.done_keywords_csv_path)
    base_df=concat([base_df, df])
    base_df=base_df.drop_duplicates(subset=['keyword'])
    base_df.to_csv(scrap_config.done_keywords_csv_path,index=False)
    
def save_result():
    final_df=DataFrame(result_list)
    date=datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name=f"report_{date}"
    final_df.to_csv(f"{file_name}.csv",index=False)
    if scrap_config.keep_record_of_keywords:
        update_done_keyword_csv(final_df)
        
def get_driver():
    url='https://www.simply.com/en/'
    driver=Chrome(executable_path=r"/home/rifat/Documents/interview/domain_scraping/driver/chromedriver")
    driver.get(url)
    return driver

def is_english_word(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        pass
    else:
        return s
    
def prepare_keyword_list(df:DataFrame):
    df.columns = map(str.lower, df.columns)
    keywords=list(df.keyword)
    keywords_set=set(map(is_english_word,keywords))
    if None in keywords_set:
        keywords_set.remove(None)
    if scrap_config.exclude_done_keywords:
        keywords_list=exclude_checked_keywords(keywords_set)
    else:
        keywords_list=list(keywords_set)
    keywords_list=[word.replace(" ","") for word in keywords_list]
    num_of_lists=20
    lists_of_keywords=[keywords_list[x:x+num_of_lists] for x in range(0, len(keywords_list), num_of_lists)]
    return lists_of_keywords

def check_domain_availability(keywords_list):
    global domain_extentions
    global result_list
    driver =get_driver()
    is_first_load=False
    for keyword in keywords_list:
        result_dict={'keyword':keyword}
        for extention in scrap_config.domain_extentions:
            try:
                full_domain=f"{keyword}.{extention}"
                inputElement = driver.find_element_by_class_name("form-control")
                if not is_first_load:
                    inputElement.send_keys(Keys.CONTROL + "a")
                    inputElement.send_keys(Keys.DELETE)
                is_first_load=False
                inputElement.send_keys(full_domain)
                inputElement.send_keys(Keys.ENTER)
                time.sleep(.6)
                elements=driver.find_elements_by_class_name('text-success')
                text=elements[0].text
                if 'The domain you searched for is available!' in text:
                    result_dict[extention]=1
                    # print(full_domain,"-->Domain Available")
                else:
                    result_dict[extention]=0
                    # print(full_domain,"-->Domain not available")
                
                del elements
            except Exception as e:
                result_dict[extention]=str(e)
                print("Exception-->",e)
        result_dict["checking_date"]=datetime.now().strftime("%Y-%m-%d %H-%M-%S")
        result_list.append(result_dict)
    driver.quit()

In [155]:
df=read_csv('google_dk__all-keywords_2022-01-23_11-32-06-2.csv',encoding="latin1")
lists_of_keywords = prepare_keyword_list(df)


Excluding already checked keywords
New Word List Length--> 185
Number of checked keywords-> 184
Number of unchecked keywords-> 185


In [51]:
done_keywords_df=read_csv(scrap_config.done_keywords_csv_path)
done_keywords_df.columns=map(str.lower,done_keywords_df.columns)
checked_keyword_set=set(done_keywords_df.keyword)
# =========
new_keywords_df=read_csv('report_2022-01-26_16-35-53.csv')
# new_keywords_df.columns=map(str.lower,done_keywords_df.columns)
new_keyword_set=set(new_keywords_df.keyword)

In [54]:
list(new_keyword_set-checked_keyword_set)

['ledlystilbil']

In [59]:
update_done_keyword_csv(read_csv("report_2022-01-26_16-35-53.csv"))

In [153]:
df=read_csv("report_2022-01-26_16-35-53.csv")
df=df[["keyword","checking_date"]]
base_df=read_csv(scrap_config.done_keywords_csv_path)
# base_df = base_df.append(df, ignore_index=True)
base_df=concat([base_df, df])
base_df=base_df.drop_duplicates(subset=['keyword'])
base_df.to_csv(scrap_config.done_keywords_csv_path,index=False)
base_df

,keyword,checking_date
0,hjemisbil,2022-01-26 16-34-14
1,hunderampetilbil,2022-01-26 16-34-16
2,billigsteleasingbil2021,2022-01-26 16-34-16
3,bilbank,2022-01-26 16-34-16
4,findbil,2022-01-26 16-34-18
...,...,...
180,legotechnicbil,2022-01-26 16-35-50
181,erhvervsleasingbil,2022-01-26 16-35-50
182,bilbasenbrugtbil,2022-01-26 16-35-51
183,leasingbilprivat,2022-01-26 16-35-53


In [19]:
t0=time.time()
with futures.ThreadPoolExecutor() as executor: # default/optimized number of threads
  titles = list(executor.map(check_domain_availability, lists_of_keywords))
save_result()
print("Time Taken-->",time.time()-t0)

/tmp/ipykernel_8350/882774213.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=Chrome(executable_path=r"/home/rifat/Documents/interview/domain_scraping/driver/chromedriver")
/tmp/ipykernel_8350/882774213.py:69: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_class_name("form-control")
/tmp/ipykernel_8350/882774213.py:77: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elements=driver.find_elements_by_class_name('text-success')


Time Taken--> 125.25432658195496


In [15]:
done_keywords_df=read_csv(scrap_config.done_keywords_csv_path)
done_keywords_df.columns=map(str.lower,done_keywords_df.columns)
checked_keyword_set=set(done_keywords_df.keyword)
a=set(['advarselslamperbil',
 'advarselslamperbilsymboler',
 'airconditionbil',
 'airconditionerbil',
 'aktivsubwooferbil',
 'amerikanerbil',
 'ansvarsforsikringbilpris',
 'automatgearbil',"sfsf",None])-checked_keyword_set

In [16]:
None in a

True

In [253]:
final_df2= final_df[['keyword', 'checking_date']]

In [254]:
final_df2.to_csv("done_kwywords.csv",index=False)

In [179]:
isEnglish("how about this one : 通 asfަ")

False